In [1]:
import pandas as pd
import numpy as np

scale = 0.25

# Domain
x = 200*scale
z = 100*scale
y_air = 20*scale
y_moon = 100*scale
y = y_moon + y_air

# Reglith Layer
y_regolith = 5*scale

# Crust Layer
y_crust = y_moon - y_regolith

# Frequency and Wavelength
f_max = 2*50e6 # Maximum Relevant Frequency for the Ricker Wavelet
c = 3e8

# Permittivity and Density
density = 2 # g/cm^3
eps_bg = 1.919**density # Background Permittivity for Moon
vel_bg = c / eps_bg**0.5

lambda_0 = c / (f_max * eps_bg**0.5)

# # Fresnel Zone - Radius
# R = 0.5 * (lambda_0 * d)**0.5
# y = 4 * R

# Grid Descretization
dx = np.round(lambda_0 / 12, 3)
dy = np.round(lambda_0 / 12, 3)
dz = np.round(lambda_0 / 12, 3)

nx = int(x / dx)
ny = int(y / dy)
nz = int(z / dz)
print(f"nx: {nx}, ny: {ny}, nz: {nz}")

# x_grid, y_grid, z_grid = np.meshgrid(x_vals, y_vals, z_vals, indexing='xy')

# Time Window
time_window = 2 * y_moon / vel_bg

# Print
print(f"Domain: {x} x {y} x {z}")
print(f"Base Permittivity of Moon: {eps_bg}")
print(f"Frequency: {f_max/2}")
print(f"Wavelength: {lambda_0}")
# print(f"Fresnel Zone Radius: {R}")
print(f"Grid: {dx} x {dy} x {dz}")
print(f"Time Window: {time_window}")


nx: 384, ny: 230, nz: 192
Domain: 50.0 x 30.0 x 25.0
Base Permittivity of Moon: 3.682561
Frequency: 50000000.0
Wavelength: 1.5633142261594581
Grid: 0.13 x 0.13 x 0.13
Time Window: 3.198333333333333e-07


In [ ]:
# Editing the Lava Tube Points to Comply with the Domain

data_all = pd.read_csv('../LavaTubeData/LiDAR_InclineCave_TUBE_TLS_points.txt', sep=' ', header=None)
# Convert to numpy arrays X, Y, Z
data_points = data_all.to_numpy()
X = (data_points[:, 0])*scale
Y = (data_points[:, 2])*scale
Z = (data_points[:, 1])*scale

# Find the minimum and maximum values of X, Y, Z, and find their index

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

# Shift all points in X, Y, Z to the center of the domain
X = X - X_min
Y = Y - Y_min
Z = Z - Z_min

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

X = X - (X_min + X_max)/2 + x/2
Y = Y - Y_max + y_moon
Z = Z - (Z_min + Z_max)/2 + z/2

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

print(f"X_min: {X_min}")
print(f"X_max: {X_max}")
print(f"Y_min: {Y_min}")
print(f"Y_max: {Y_max}")
print(f"Z_min: {Z_min}")
print(f"Z_max: {Z_max}")

data_all_shifted = pd.DataFrame(np.column_stack((X, Y, Z)))
data_all_shifted = data_all_shifted.to_numpy()

# Save the new points to vtk
from pyevtk.hl import pointsToVTK
pointsToVTK("LiDAR_InclineCave_TUBE_adjusted", X, Y, Z)

X_min: 8.756347625007038
X_max: 41.24365237499296
Y_min: 18.63000500000004
Y_max: 25.0
Z_min: 4.682617124984972
Z_max: 20.317382875015028


'f:\\Projects\\LunarLeaper\\LLGPR\\src\\LiDAR_InclineCave_TUBE_adjusted.vtu'

In [ ]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator

permittivity_values = 7.6* np.ones_like(X)

In [ ]:
#Grid
# x_val = np.linspace(0, x, 5)
# z_val = np.linspace(0, z, 5)
# y_val = np.linspace(0, y_moon, 5)
x_val = np.linspace(0, x, int(nx*scale))
z_val = np.linspace(0, z, int(ny*scale))
y_val = np.linspace(0, y_moon, int(nz*scale))

In [ ]:
meshgrid_x, meshgrid_y, meshgrid_z = np.meshgrid(x_val, y_val, z_val, indexing='ij')
points = np.vstack((meshgrid_x.flatten(), meshgrid_y.flatten(), meshgrid_z.flatten())).T

# Create Delaunay triangulation
tri = Delaunay(np.column_stack((X, Y, Z)))

In [ ]:
pointsToVTK("3_Box", meshgrid_x.flatten(), meshgrid_y.flatten(), meshgrid_z.flatten())

'f:\\Projects\\LunarLeaper\\LLGPR\\src\\Box.vtu'

In [ ]:
# Check which points are within the convex hull
# Check for interpolation
simplex_indices = tri.find_simplex(points)
mask = simplex_indices >= 0

In [ ]:
# x, y, z for just the mask
x_mask = meshgrid_x.flatten()[mask]
y_mask = meshgrid_y.flatten()[mask]
z_mask = meshgrid_z.flatten()[mask]
points_cave_edge = np.vstack((x_mask, y_mask, z_mask)).T
print(f"Number of Points in Convex Hull: {len(z_mask)}")
print(f"Number of Points in Total: {len(points_cave_edge)}")

Number of Points in Convex Hull: 3997
Number of Points in Total: 3997


In [ ]:
# Defining the Minerals and Elements
# Order: Electrical Permittivity, Electrical Conductivity, Magnetic Permeability, Magnetic Loss, 
# Weight Fraction for Regolith, Weight Fraction for Crust, Density (g/cm^3)
eps_0 = 8.854e-12
mu_0 = 4*np.pi*1e-7
FeO = np.array([50, 37*eps_0, 1, 0.1*mu_0, 0.154, 0.205, 5.74])
TiO2 = np.array([20, 80*eps_0, 1, 0.1*mu_0, 0.027, 0.035, 4.23])
Al2O3 = np.array([10, 9*eps_0, 1, 0.1*mu_0, 0.145, 0.0825, 3.95])
MgO = np.array([5, 9*eps_0, 1, 0.1*mu_0, 0.1, 0.0875, 3.58])
SiO2 = np.array([3.58, 0.001*eps_0, 1.05, 0.02*mu_0, 0.44, 0.445, 2.65])
CaO = np.array([5, 6*eps_0, 1, 0.1*mu_0, 0.105, 0.1, 3.35])
Potassium = np.array([5, 4*eps_0, 1, 0.1*mu_0, 0.01, 0.01, 2.34])
Thorium = np.array([5, 4*eps_0, 1, 0.1*mu_0, 0.01, 0.01, 11.72])

# Defining the Volume Fractions for Regolith from the weight fractions and density
wt_reg = np.array([FeO[4], TiO2[4], Al2O3[4], MgO[4], SiO2[4], CaO[4]])
densities = np.array([FeO[-1], TiO2[-1], Al2O3[-1], MgO[-1], SiO2[-1], CaO[-1]])
vol_fr_individual = wt_reg / densities
vol_fr_regolith = vol_fr_individual / np.sum(vol_fr_individual)

# Defining the Volume Fractions for Crust from the weight fractions
wt_crust = np.array([FeO[5], TiO2[5], Al2O3[5], MgO[5], SiO2[5], CaO[5]])
vol_fr_individual = wt_crust / densities
vol_fr_crust = vol_fr_individual / np.sum(vol_fr_individual)

In [ ]:
# Defining the Bruggerman's Mixing Model
from scipy.optimize import fsolve
def bruggerman_mixing_model(volume_fractions, epsilon_values):
    def equation(epsilon_eff):
        numerator = 0
        denominator = 0
        for i in range(len(volume_fractions)):
            numerator += (epsilon_values[i] - epsilon_eff) * volume_fractions[i]
            denominator += (epsilon_values[i] + 2 * epsilon_eff) * volume_fractions[i]
        return numerator / denominator
    
    epsilon_eff = fsolve(equation, epsilon_values[0])
    return epsilon_eff[0]

epsilon_values = [FeO[0], TiO2[0], SiO2[0], Al2O3[0], CaO[0], MgO[0], Potassium[0], Thorium[0]]
epsilon_regolith = bruggerman_mixing_model(vol_fr_regolith, epsilon_values)
mu_regolith = np.mean([FeO[2], TiO2[2], SiO2[2], Al2O3[2], CaO[2], MgO[2], Potassium[2], Thorium[2]])
sigma_e_regolith = np.mean([FeO[1], TiO2[1], SiO2[1], Al2O3[1], CaO[1], MgO[1], Potassium[1], Thorium[1]])
sigma_m_regolith = np.mean([FeO[3], TiO2[3], SiO2[3], Al2O3[3], CaO[3], MgO[3], Potassium[3], Thorium[3]])

epsilon_crust = bruggerman_mixing_model(vol_fr_crust, epsilon_values)
mu_crust = mu_regolith
sigma_e_crust = sigma_e_regolith
sigma_m_crust = sigma_m_regolith

In [11]:
with open('7_moon_true_heterogeneous_tube_filled.in', 'w') as f:

    # Write the domain and discretization
    f.write(f"#domain: {x} {y} {z}\n")
    f.write(f"#dx_dy_dz: {dx} {dy} {dz}\n")
    f.write(f"#time_window: {time_window}\n")

    # Define the GPR source
    f.write("#waveform: ricker 1 50e6 ricker_wavelet\n")
    f.write(f"#hertzian_dipole: z {x/2 - 5} {y_moon} {z/2} ricker_wavelet\n")
    
    f.write(f"#material: {epsilon_regolith} {sigma_e_regolith} {mu_regolith} {sigma_m_regolith} moon_regolith_material\n")
    f.write(f"#fractal_box: 0 {y_crust} 0 {x} {y_moon} {z} 2.5 1 1 1 1 moon_regolith_material moon_regolith\n")

    f.write(f"#material: {epsilon_crust} {sigma_e_crust} {mu_crust} {sigma_m_crust} moon_crust_material\n")
    f.write(f"#fractal_box: 0 0 0 {x} {y_crust} {z} 2.5 1 1 1 1 moon_crust_material moon_crust\n")

    eps_r_value = 7.6
    sigma_value = 1e-10
    mu_r_value = 1.0
    sigma_m_value = 0.0
    f.write(f"#material: {eps_r_value} {sigma_value} {mu_r_value} {sigma_m_value} moon_lava_tube \n")
    # Dielectric smoothing needs to be turned off, add higher discretization for numerical stability

    # Loop over each cell to define the materials
    for i in range(len(x_mask)):
        # Get the dielectric properties for the current cell
        f.write(f"#box: {x_mask[i]} {y_mask[i]} {z_mask[i]} {x_mask[i]+dx} {y_mask[i]+dy} {z_mask[i]+dz} moon_lava_tube n\n")

    f.write(f"#cylinder: {x/2 - 5} {y/2-10} {x/2+5} {y/2-10} {y/2-10} {z/2+5} 5 pec\n")

    f.write(f"#rx: {x/2 + 5} {y_moon} {z/2}\n")
    f.write(f"#geometry_view: 0 0 0 {x} {y} {z} {dx} {dy} {dz} 7_moon_true_heterogeneous_tube_filled n \n")

# Output success message
print("gprMax input file '7_moon_true_heterogeneous_tube_filled.in' generated.")

gprMax input file 'moon_lava_base_n4_2.in' generated.
